# Esperienza di Fletcher

*Sulla base della lettura di un file CSV contenente le masse traenti, l'accelerazione rilevata e la massa del carrello, calcola i coefficienti della retta di regressione, le incertezze dei parametri e il coeffiente di correlazione utilizzando la funzione `linregress` del pacchetto SciPy.
Visualizza i risultati in un grafico a dispersione e fornisce una tabella di raffronto tra l'accelerazione rilevata e quella prevista.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
# funzione utile per formattare la tabella finale
from tabulate import tabulate

In [ ]:
nome_file = input("Inserire il nome del file CSV: ")
# il file Fletcher.csv riporta i dati in tre colonne
file_in = open(nome_file, "r")
coppie_dati = np.loadtxt(file_in, delimiter = ",", comments = '#', usecols = (0,1,2))

Viene utilizzata la funzione `linregress()` della libreria SciPy. Riportati i risultati nella tuple `esiti` dalla quale vengono estratti gli elementi ``slope`` e ``intercept``.

In [ ]:
n = len(coppie_dati)

nparrayX_Y = coppie_dati.transpose()
dati_x = nparrayX_Y[0]
dati_y = nparrayX_Y[1]
dati_M = nparrayX_Y[2]
file_in.close()

esiti = linregress(dati_x,dati_y)

a = esiti.slope
b = esiti.intercept

Qui si estraggono le stime delle deviazioni standard e il coefficiente di correlazione.

In [ ]:
previsioni_y = a*dati_x + b
residui = dati_y - previsioni_y

sigma = np.std(residui, ddof=2)
sigma_a = esiti.stderr
sigma_b = esiti.intercept_stderr
coeff_correlazione = esiti.rvalue

In [ ]:
min_x = min(dati_x)
max_x = max(dati_x)
min_y = min(dati_y)
max_y = max(dati_y)
delta_x = (max_x-min_x)/15
delta_y = (max_y-min_y)/15

Si costruiscono le rette che individuano la fascia di incertezza. 

In [ ]:
x = np.linspace(min_x-delta_x, max_x+delta_x, 100)
y = a*x + b
y_sup = (a+sigma_a)*x + (b+sigma_b)
y_inf = (a-sigma_a)*x + (b-sigma_b)

In [ ]:
massimo_residui = max(abs(residui))

if a>=0:
    posizione_txt = max_x - 4*delta_x
elif a<0:
    posizione_txt = min_x + delta_x 

Dispersione dei dati, retta di regressione. La fascia di incertezza non appare data l'esiguità dell'errore.

In [ ]:
plt.grid(which = 'both', color = '.85', linestyle = '-', linewidth=1)
plt.vlines(dati_x, dati_y-sigma, dati_y+sigma, linewidth = .5, color = 'b')
plt.fill_between(x, y_inf, y_sup, alpha = .1, linewidth = 0, color='r')
plt.plot(x, y, color = 'red', linewidth = 2, label = 'retta di regressione')
plt.scatter(dati_x, dati_y, s = 20, c = 'blue', zorder = 3, label = 'dati rilevati')
plt.title("Proporzionalità tra accelerazione e masse traenti")
plt.xlabel('massa traente (g)')
plt.ylabel('accelerazione (m/s^2)')
plt.text(posizione_txt-delta_x, delta_y, 'y = ({0:6.4f})x + ({1:6.3f})\n a = {0:6.4f} $\pm$ {2:6.4f}\n b = {1:6.3f} $\pm$ {3:6.3f}\n r = {4:6.3f}'.format(a, b, sigma_a, sigma_b, coeff_correlazione), c = 'r')
plt.legend()
plt.show()

Fissato il valore dell'accelerazione di gravità, si calcola l'accelerazione prevista in base alle masse coinvolte in ciascuna prova e quindi si confronta quella rilevata con quella prevista.

In [ ]:
g = 9.81
acc_prev = g/(dati_x+dati_M)*dati_x
delta_acc = dati_y - acc_prev

Utilizzando la funzione di libreria ``tabulate`` si costruisce la tabella finale di raffronto.

In [ ]:

tabella = np.transpose([dati_M, dati_x, dati_y, acc_prev, delta_acc ])
print(tabulate(tabella,headers=["massa carrello (g)","massa traente: m_t (g)","acc.rilevata: a_r (m/s^2)","acc.prevista: a_p (m/s^2)","a_r-a_p (m/s^2)"],floatfmt=(".1f",".1f",".4f",".4f",".2f")))